In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pip install plotly==4.10.0

     |████████████████████████████████| 13.0MB 236kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
#pd.options.plotting.backend = "plotly"
df = pd.DataFrame(dict(a=[1, 2, 3], b=[2, 4, 6]))
df1=pd.DataFrame(dict(c=[1, 2, 3], d=[2, 4, 6]),index=[9,5,8])
pd.concat([df,df1],axis=1)

,a,b,c,d
0,1.0,2.0,NaN,NaN
1,2.0,4.0,NaN,NaN
2,3.0,6.0,NaN,NaN
5,NaN,NaN,2.0,4.0
8,NaN,NaN,3.0,6.0
9,NaN,NaN,1.0,2.0


In [ ]:
data=pd.read_csv("/content/drive/My Drive/Sentiment_combined.csv")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
df_analysis_1=data.dropna()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
import nltk
nltk.download('stopwords') 
stopwords=nltk.corpus.stopwords.words('english')
custom_stopwords=['rt','amp']
stopwords.extend(custom_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_analysis_1['Review'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())

In [ ]:
df_analysis_1['preprocess']=preprocessed_synopsis

In [ ]:
df_analysis_1.head(3)

,Index,Source,Date,Name,Hotel Key,Rating,Summary,Review Type,Review,Review_sentiment_polarity,Review_sentiment_Label,Summary_sentiment_polarity,Summary_sentiment_Label,preprocess
0,0,Booking.com,25-09-18,Hilton Auckland,1,10.0,The staff made sure our stay was one of a kind...,Negative_Review,Bigger pool.,0.0000,Neutral,0.6908,Positive,bigger pool
1,1,Booking.com,25-09-18,Hilton Auckland,1,6.3,An ok hotel but not where I would stay for a s...,Negative_Review,The hotel design is functional rather than hig...,0.4086,Positive,0.6310,Positive,hotel design functional rather high end balcon...
2,2,Booking.com,23-09-18,Hilton Auckland,1,7.1,Location was fab.,Negative_Review,Tiled shower had blacken grout and silicon. No...,0.4404,Positive,0.4588,Positive,tiled shower blacken grout silicon would expec...


In [ ]:
food=['food','delicious','taste','beverage','yum','yummy', 'bar', 'drink', 'vegetarian', 'gustodelicious', 'gourmet', 'restuarant','pizza','dish', 'cooke','breakfast','replinished','fooddrink', 'vegan','kitchen','cinnamon','gluten','toast','tasty','fry','sauce','chief','soup', 'dinner','margherita','milk','buffet','pepper','garlic']
room=['room','spacious']
staff=['staff', 'friendly', 'helpful','staff','professional',"service",'customer','polite', 'reception','friendliness','unhelpful','rude']
bed=['bed','comfy','king','pillow','linen','sleep','slept']
bathroom=['bathroom','clean','shower','toilet','wet','tiles','bath','makeup','hang','water','toiletry','bath','sink']
cleanliness=['clean','tidy','neat','dust','smelled','spotless']
space=['space','spacious','big','small','area']
facilities=['spa','gym','swimming','steam','sauna','wifi','pool','parking','park','unlimited','amenities','gb','facility']
location=['location','view','central','city','auckland','convenient','shopping','harbour']
price=['price','paid','paying','tax','value','fair','reasonable','pricy']

In [ ]:
pattern_food = '|'.join(food)
pattern_room = '|'.join(room)
pattern_staff = '|'.join(staff)
pattern_bed = '|'.join(bed)
pattern_bathroom = '|'.join(bathroom)
pattern_cleanliness = '|'.join(cleanliness)
pattern_space = '|'.join(space)
pattern_facilities = '|'.join(facilities)
pattern_location = '|'.join(location)
pattern_price = '|'.join(price)

In [ ]:
df_analysis_1["Food"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_food))
df_analysis_1["Room"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_room))
df_analysis_1["Staff"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_staff))
df_analysis_1["Bed"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_bed))
df_analysis_1["Bathroom"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_bathroom))
df_analysis_1["Cleanliness"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_cleanliness))
df_analysis_1["Space"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_space))
df_analysis_1["Facilities"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_facilities))
df_analysis_1["Location"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_location))
df_analysis_1["Price"]=pd.DataFrame(df_analysis_1.Review.str.contains(pattern_price))

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(df_analysis_1,test_size=0.33)

In [ ]:
from keras.preprocessing.text import Tokenizer
vect=Tokenizer()
vect.fit_on_texts(train['preprocess'])
vocab_size = len(vect.word_index) + 1

In [ ]:
from keras.preprocessing.sequence import pad_sequences
encoded_docs_train = vect.texts_to_sequences(train['preprocess'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=1200, padding='post')
print(padded_docs_train)

[[ 144    1  862 ...    0    0    0]
 [ 211 3117    3 ...    0    0    0]
 [  10    2    4 ...    0    0    0]
 ...
 [  19  240   46 ...    0    0    0]
 [  27   23   51 ...    0    0    0]
 [5817 5818  140 ...    0    0    0]]


In [ ]:
encoded_docs_test = vect.texts_to_sequences(test['preprocess'])
max_length = vocab_size
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1200, padding='post')

In [ ]:
from keras.layers import Dense, Flatten, LSTM, Conv1D, Conv2D, MaxPooling1D, Dropout, Activation,GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.optimizers import Adam
from time import time
from keras.callbacks import TensorBoard
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam

from keras.layers.embeddings import Embedding
from keras.callbacks import ReduceLROnPlateau

In [ ]:
model = Sequential()
#model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(LSTM(512,return_sequences=True))
model.add(LSTM(256,return_sequences=True))
#model.add(Dropout(0.5))
model.add(LSTM(128,return_sequences=True))
#model.add(Dropout(0.15))
model.add(LSTM(64))
#model.add(GlobalMaxPool1D())
model.add(Dense(11, activation='sigmoid'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1200, 50)          283800    
_________________________________________________________________
lstm_14 (LSTM)               (None, 1200, 512)         1153024   
_________________________________________________________________
lstm_15 (LSTM)               (None, 1200, 256)         787456    
_________________________________________________________________
lstm_16 (LSTM)               (None, 1200, 128)         197120    
_________________________________________________________________
lstm_17 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_5 (Dense)              (None, 11)                715       
Total params: 2,471,523
Trainable params: 2,471,523
Non-trainable params: 0
____________________________________________

In [ ]:
y_train=train[['Review Type','Food', 'Room', 'Staff', 'Bed','Bathroom', 'Cleanliness', 'Space', 'Facilities', 'Location', 'Price']]
y_train['Review Type']=y_train['Review Type'].map({'Negative_Review':0,'Positive_Review':1})
y_train=y_train.applymap(lambda x: 1 if x==True else 0 )

In [ ]:
np.array(y_train.head(2))

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]])

In [ ]:
y_train

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Conv1D(64, 3, activation='sigmoid'))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Conv1D(100, 3, activation='sigmoid'))
# model.add(Dropout(0.70))
model.add(Conv1D(48, 3, activation='sigmoid'))
model.add(Flatten())
model.add(Dense(11))

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 1200, 71)          402996    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1198, 64)          13696     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1196, 100)         19300     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1194, 100)         30100     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1192, 48)          14448     
_________________________________________________________________
flatten_1 (Flatten)          (None, 57216)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 11)              

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 71, input_length=1200))
#model.add(Dropout(0.5))
model.add(Conv1D(300, 3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.70))
# model.add(Conv1D(200, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
model.add(Dense(11))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=["AUC",'accuracy',f1_m,precision_m, recall_m,hn_multilabel_loss,hamming_loss])
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4)]

history = model.fit(padded_docs_train, y_train,
                    epochs=300,
                    batch_size=128,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/300
42/42 [==============================] - 5s 129ms/step - loss: 0.0635 - auc: 0.9951 - accuracy: 0.5111 - f1_m: 0.9523 - precision_m: 0.9671 - recall_m: 0.9383 - hn_multilabel_loss: 0.7023 - hamming_loss: 0.0346 - val_loss: 0.1208 - val_auc: 0.9864 - val_accuracy: 0.4829 - val_f1_m: 0.9170 - val_precision_m: 0.9044 - val_recall_m: 0.9301 - val_hn_multilabel_loss: 1.3422 - val_hamming_loss: 0.0464
Epoch 2/300
42/42 [==============================] - 5s 116ms/step - loss: 0.0593 - auc: 0.9960 - accuracy: 0.5064 - f1_m: 0.9525 - precision_m: 0.9639 - recall_m: 0.9416 - hn_multilabel_loss: 0.6485 - hamming_loss: 0.0343 - val_loss: 0.1240 - val_auc: 0.9866 - val_accuracy: 0.4898 - val_f1_m: 0.9152 - val_precision_m: 0.9036 - val_recall_m: 0.9272 - val_hn_multilabel_loss: 1.3776 - val_hamming_loss: 0.0460
Epoch 3/300
42/42 [==============================] - 5s 117ms/step - loss: 0.0611 - auc: 0.9957 - accuracy: 0.5060 - f1_m: 0.9516 - precision_m: 0.9657 - recall_m: 0.9381 - hn_mu

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def hn_multilabel_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

def hamming_loss(y_true, y_pred):
        return K.mean(y_true*(1-y_pred)+(1-y_true)*y_pred)